In [1]:
# 简直拉跨，否了
import requests
header = {
    'Accept': 'text/html',
    'Accept-Encoding': 'utf-8',
    'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'Connection': 'keep-alive',
    'Cookie': '',
    'DNT': '1',
    'Host': '219.216.96.4',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/131.0.0.0 Safari/537.36 Edg/131.0.0.0'
}
response = requests.get('https://pass.neu.edu.cn/')
content=response.content
print(content)

In [2]:
import pandas as pd

course_list = pd.read_html(open(r'files\aaa.xls', 'r', encoding='utf-8').read())[-2].values.tolist()

for course in course_list:
    print(course)

['第一节', nan, '商务数据管理与信息系统 (A077810)  (俞竹超)  (10-17 1号A302(浑南校区))', '大学英语㈢ (A080963)  (刘红)  (3-16 1号B611-语音室(浑南校区))', '信息管理专业概论 (A077901)  (樊治平)  (1-8 1号B111(浑南校区))  组织行为学 (A078002)  (王艳梅)  (10-17 信息A101(浑南校区))', '商务数据管理与信息系统 (A077810)  (俞竹超)  (10-17 1号A302(浑南校区))', nan, nan]
['第二节', nan, '商务数据管理与信息系统 (A077810)  (俞竹超)  (10-17 1号A302(浑南校区))', '大学英语㈢ (A080963)  (刘红)  (3-16 1号B611-语音室(浑南校区))', '信息管理专业概论 (A077901)  (樊治平)  (1-8 1号B111(浑南校区))  组织行为学 (A078002)  (王艳梅)  (10-17 信息A101(浑南校区))', '商务数据管理与信息系统 (A077810)  (俞竹超)  (10-17 1号A302(浑南校区))', nan, nan]
['第三节', nan, '体育(三) (A082079)  (王儒轩)  (3-14 )', '概率论与数理统计 (A077899)  (张驰)  (1-16 文管A227(浑南校区))', '数据库应用开发技术 (A077900)  (许博)  (1-6 1号A204(浑南校区))', '概率论与数理统计 (A077899)  (张驰)  (1-8,10-13 文管A129(浑南校区))', '大学英语㈢ (A080963)  (刘红)  (3-8,10-17 1号B111(浑南校区))', nan]
['第四节', nan, '体育(三) (A082079)  (王儒轩)  (3-14 )', '概率论与数理统计 (A077899)  (张驰)  (1-16 文管A227(浑南校区))', '数据库应用开发技术 (A077900)  (许博)  (1-6 1号A204(浑南校区))', '概率论与数理统计 (A077899)  (张驰)  (1-8,10-13 文管A129(浑

In [1]:
import smtplib
from email.mime.text import MIMEText
from email.header import Header

mail_info = open('data\mail_key', 'r', encoding='utf-8').read().split('\n')
from_addr = mail_info[0]
password = mail_info[1]
to_addr = '20231162@stu.neu.edu.cn'
smtp_server = mail_info[2]

head="hhh"
text="dsjakfl;dshajgjdkahuiuiicxzhv"

msg = MIMEText(text,'plain','utf-8')
 
# 邮件头信息
msg['From'] = Header(from_addr)
msg['To'] = Header(to_addr)
msg['Subject'] = Header(head)
 
# 开启发信服务，这里使用的是加密传输
server=smtplib.SMTP_SSL(smtp_server)
server.connect(smtp_server,465)
server.login(from_addr, password)
server.sendmail(from_addr, to_addr, msg.as_string())
server.quit()

(221, b'Bye')

In [ ]:
from flask import Flask, request, redirect, url_for, render_template, make_response
import hashlib

app = Flask(__name__)

def generate_sha256(username):
    """生成用户名的 SHA-256 值"""
    return hashlib.sha256(username.encode()).hexdigest()

@app.route('/')
def index():
    return redirect(url_for('login'))

@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == 'POST':
        # 获取用户名
        username = request.form.get('username')
        if username:
            sha256_username = generate_sha256(username)

            # 检查 Cookie 中是否存在 SHA-256 值
            if request.cookies.get('user_hash') == sha256_username:
                return redirect(url_for('success'))
            else:
                return redirect(f'/warning/{username}')

    return '''<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>登录</title>
</head>
<body>
    <h1>登录页面</h1>
    <form method="POST">
        <label for="username">用户名：</label>
        <input type="text" id="username" name="username" required>
        <input type="submit" value="登录">
    </form>
</body>
</html>
'''

@app.route('/success')
def success():
    return "成功登录！欢迎回来！"

@app.route('/warning/<username>', methods=['GET', 'POST'])
def warning(username):
    if request.method == 'POST':
        sha256_username = generate_sha256(username)
        response = make_response(redirect(url_for('success')))
        # 设置 Cookie，增加安全性
        response.set_cookie(
            'user_hash', 
            sha256_username, 
            max_age=60 * 1,  # 设置 Cookie 有效期为 1 分钟
            httponly=True,   # 防止通过 JavaScript 访问 Cookie
            secure=True,     # 仅在 HTTPS 下传输 Cookie
            samesite='Lax'   # 防止 CSRF 攻击
        )
        return response
    return f'''<!DOCTYPE html>
    <html lang="en">
    <head><meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>警告</title>
    </head>
    <body>
    <h1>警告</h1>
    <p>你正在使用一个新设备登陆，请确认你是本人！</p>
    <p>用户名：{username}</p>
    <form method="POST"><input type="submit" value="确认"></form>
    </body></html>'''

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=8080, debug=True)
